In [17]:
# Import neccessary libraries
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2
import os
import traceback

In [18]:
# create an HandLandmarker object.
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the image mode:
hand_landmarker_options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='models/hand_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE,
    num_hands=1,
)


In [19]:
#Create an ObjectDetector object.
ObjectDetector = mp.tasks.vision.ObjectDetector
ObjectDetectorOptions = mp.tasks.vision.ObjectDetectorOptions


detector_options = ObjectDetectorOptions(
    base_options=BaseOptions(model_asset_path='models/efficientdet.tflite'),
    score_threshold=0.5,
    max_results=3,
    running_mode=VisionRunningMode.IMAGE,
    category_denylist=["person"],
)


In [20]:

landmarker = HandLandmarker.create_from_options(hand_landmarker_options) 
detector = vision.ObjectDetector.create_from_options(detector_options)

In [21]:
from onlaweng_utils import draw_bounding_boxes, draw_landmarks

In [24]:
WIDTH = 640
HEIGHT = 480

In [71]:
import math
class Line2D :
    def __init__(self, p1, p2) :
        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]
        m = dy / (dx if dx != 0 else 0.000001)
        m = m if m != 0 else 0.000001
        c = tip_landmark[1] - (m * tip_landmark[0])
        
        self.x_top_intercept = round(-c/m)
        self.x_bot_intercept = round((HEIGHT - c)/m)
        self.y_left_intercept = round(c)
        self.y_right_intercept = round((m * WIDTH) + c)
        
        self.m = m
        self.c = c
        
    # draw lines from given point to each intercepts
    def draw_intercept(self, image, pt) :
        h, w, _ =  image.shape
        if self.x_top_intercept >= 0 and self.x_top_intercept < w :
            cv2.line(image, pt, (self.x_top_intercept, 0), (0,222,0), 1)
        if self.x_bot_intercept >= 0 and self.x_bot_intercept < w :
            cv2.line(image, pt, (self.x_bot_intercept, h), (0,222,0), 1)
        if self.y_left_intercept >= 0 and self.y_left_intercept < h :
            cv2.line(image, pt, (0, self.y_left_intercept), (0,222,0), 1)
        if self.y_right_intercept >= 0 and self.y_right_intercept < h :
            cv2.line(image, pt, (w, self.y_right_intercept), (0,222,0), 1)
    
    # calculate distance between given point and this line
    def pt_distance(self, pt) :
        A = self.m
        B = -1
        C = self.c
        
        return math.fabs((A * pt[0]) + (B * pt[1]) + C) / math.sqrt((A**2) + (B**2))
    
def pt_pt_distance(pt1, pt2):
    x1, y1 = pt1
    x2, y2 = pt2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

In [72]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    mp_img = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    
    # detect hands and objects
    detector_results = detector.detect(mp_img)
    hand_results = landmarker.detect(mp_img)
    
    # # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    #print(results)
    try :
        if hand_results.hand_landmarks and hand_results.handedness[0][0].category_name == 'Right':
            tip_landmark = hand_results.hand_landmarks[0][8]
            dip_landmark = hand_results.hand_landmarks[0][7]
            tip_landmark = (round(tip_landmark.x * WIDTH), round(tip_landmark.y * HEIGHT))
            dip_landmark = (round(dip_landmark.x * WIDTH), round(dip_landmark.y * HEIGHT))
            
            l = Line2D(tip_landmark, dip_landmark)
            l.draw_intercept(image, tip_landmark)
            
            draw_landmarks(image, hand_results.hand_landmarks[0], mp.solutions.hands.HAND_CONNECTIONS)
        
            pointing_threshold = 100
            if detector_results :
                for idx, detection in enumerate(detector_results.detections) :
                    bbox = detection.bounding_box
                    centroid_x = bbox.origin_x + (bbox.width // 2)
                    centroid_y = bbox.origin_y + (bbox.height // 2)
                    centroid = (centroid_x, centroid_y)
                    
                    if l.pt_distance(centroid) <= pointing_threshold and pt_pt_distance(centroid, dip_landmark) > pt_pt_distance(centroid, tip_landmark) :
                        draw_bounding_boxes(image, detection, 2023+idx)
                        cv2.circle(image, centroid, 10, (0, 255, 0), 2)
    except Exception as e:
        print(traceback.format_exc())
        break


    cv2.imshow('MediaPipe Object Detection', image)
    if cv2.waitKey(5) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# 